In [85]:
import pandas as pd
import numpy as np
import datetime
import json
import pivottablejs
from IPython.display import HTML
import IPython.core.display as di
from IPython.display import display
from IPython.display import Markdown as md

In [114]:
#!pip install --user pyarrow

     |████████████████████████████████| 64.2MB 27.8MB/s eta 0:00:01     |███████████████████████████▉    | 55.9MB 27.8MB/s eta 0:00:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [86]:
df = pd.read_csv('metadata/Results-20180101.csv', low_memory=False, sep=',')

We should group sequences based on their Genogroups, and there are 4 columns for this info

In [87]:
s = set()
for i in range(1,5):
    s = s.union(set(df['seq %d: Genus / Genogroup'%i].unique()))
s

{'Caliciviridae Norovirus GI',
 'Caliciviridae Norovirus GII',
 'Caliciviridae Norovirus GIII',
 'Caliciviridae Norovirus GIV',
 'Caliciviridae Norovirus GVII',
 'Unassigned',
 nan}

In [88]:
available_genogroups = list(s)
available_genogroups.remove(np.nan)
available_genogroups.remove('Unassigned')
available_genogroups

['Caliciviridae Norovirus GVII',
 'Caliciviridae Norovirus GIV',
 'Caliciviridae Norovirus GIII',
 'Caliciviridae Norovirus GII',
 'Caliciviridae Norovirus GI']

In [89]:
for i in range(1,5):
    gengroup = 'seq %d: Genus / Genogroup'%i
    #print(df[gengroup].fillna().sum())
    df[df[gengroup].isna()][gengroup] = 'Unassigned'

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


#### We will create a new DataFrame, but each seqX will be in a separate row
For that we need to determine, which columns/metadata will be copied along

In [90]:
#df.columns

In [91]:
#First get rid of the columns that contain a summary of other columns
drop_cols = ['ORF1', 'ORF1 variant', 'ORF2', 'ORF2 variant', 'Genus / Genogroup' ]
df = df.drop(drop_cols, axis=1)

In [92]:
copycols = list(df.columns)
#print(copycols)
copycols = ['User', 'Institute', 'Database ID', 'Country', 'Submission Date','Last Update', 'Outbreak Nr', 'Sample Date', 'Dutch lab Y/N', 'Nosocomial infection Y/N (sampledate > 2days after date of hospitalization)', 'Outbreak or sporadic case O/S',
 'Source of the sample', 'Specify source', 'Specify animal', 'Suspected country of infection', 'Date of first disease', 'Geo-coded location', 'Suspected mode of transmission',
 'Specify other mode of transmission', 'Food item', 'Specify food item', 'Setting of the outbreak', 'Specify setting', 'Point source transmission Y/N', 'Date of point source transmission',
 'Nr of persons affected', 'Nr of persons at risk', 'Nr of cases deceased', 'Nr of cases hospitalized due to infection', 'Nr of cases of ages 0 to 4', 'Nr of cases of ages 5 to 14',
 'Nr of cases of ages 15 to 64', 'Nr of cases of age 65 or older', 'Nr of cases with vomiting', 'Nr of cases with diarrhea', 'Nr of cases with vomiting AND diarrhea',
 'Mixed infection Y/N', 'Specify other pathogen(s)', 'Nr of samples tested', 'Nr of PCR positive samples', 'Nr of PCR negative samples', 'Included in II.4 P2 capsid surveillance',
 'fasta_id', 'reference_id', 'fragment_begin', 'Genus-Genogroup', 'ORF1', 'ORF1_variant', 'ORF2', 'ORF2_variant', 'Reference_sequence_for_positions', 'Cluster']

In [179]:
tdfs = []
for si in range(1,5):
    rename_dict = {'seq %d: fasta id'%(si): 'fasta_id',
                   'seq %d: reference id'%(si): 'reference_id',
                   'seq %d: fragment begin'%(si): 'fragment_begin',
                   'seq %d: fragment end'%(si): 'fragment_end',
                   'seq %d: Genus / Genogroup'%(si): 'Genus-Genogroup',
                   'seq %d: ORF1'%(si): 'ORF1',
                   'seq %d: ORF1 variant'%(si): 'ORF1_variant',
                   'seq %d: ORF2'%(si): 'ORF2',
                   'seq %d: ORF2 variant'%(si): 'ORF2_variant',
                   'seq %d: Reference sequence for positions'%(si): 'Reference_sequence_for_positions',
                   'seq %d: Cluster'%(si): 'Cluster'}
    tdfs.append(df.rename(rename_dict, axis=1)[copycols])
#     for ir, r in tdf.iterrows():
#         gengroup = 'Genus-Genogroup'
#         if r[gengroup] in available_genogroups:
#             ndf.append(r, ignore_index=True)

ndf = pd.concat(tdfs)

In [180]:
print(ndf.shape)
ndf = ndf[ndf['Genus-Genogroup'].notna()]
ndf = ndf.reset_index(level=0, drop=True)
print(ndf.shape)
print(ndf[ndf['Genus-Genogroup']=='Unassigned'].shape)

(8772, 52)
(3140, 52)
(27, 52)


## Data cleaning
drop columns with irrelevant data

Get rid of columns with no value

In [181]:
for c in ndf.columns:
    if ndf[c].notna().sum()==0:
        print("Dropping column: %s"%c)
        ndf.drop(c, axis=1, inplace=True)
    

Dropping column: Nosocomial infection Y/N (sampledate > 2days after date of hospitalization)
Dropping column: Specify animal


In [182]:
drop_irrelevant_cols = ['Dutch lab Y/N',  'Nr of cases with vomiting AND diarrhea']
ndf.drop(drop_irrelevant_cols, axis=1, inplace=True)

In [183]:
# Looks better this way
colname = 'Suspected country of infection'
ndf[ndf[colname]=='Same as reporting country'][colname] = ndf[ndf[colname]=='Same as reporting country']['Country']

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


#### split ORFs into main and subtype

In [192]:
ndf['ORF1'].unique()

array(['GII.P16', nan, 'GII.P4', 'GII.P17', 'GII.Pe', 'GII.P21', 'GI.P2',
       'GI.Pb', 'GI.P4', 'GI.P3', 'GII.P7', 'Unknown', 'GII.P2', 'GII.Pc',
       'GI.P1', 'GII.P8', 'GI.P7', 'GII.P12', 'GI.P5', 'GI.P6', 'GI.Pd',
       'GI.Pa', 'GII.Pq', 'GII.Pg', 'GII.P11', 'GII.P22', 'GII.P20',
       'GII.P30 (GII.Pc)', 'GI.P13 (GI.Pd)', 'GII.P31 (GII.Pe)', 'GI.P9'],
      dtype=object)

In [191]:
ndf.loc[ndf['ORF1']=='Could not assign', 'ORF1'] = 'Unknown'
ndf.loc[ndf['ORF2']=='Could not assign', 'ORF2'] = 'Unknown'

In [193]:
ndf['ORF1_sub'] = ndf['ORF1'].apply(lambda x: x.split(".")[-1] if type(x) != np.float else 'Unknown')
ndf['ORF1'] = ndf['ORF1'].apply(lambda x: x.split(".")[0] if type(x) != np.float else 'Unknown')
ndf['ORF2_sub'] = ndf['ORF2'].apply(lambda x: x.split(".")[-1] if type(x) != np.float else 'Unknown')
ndf['ORF2'] = ndf['ORF2'].apply(lambda x: x.split(".")[0] if type(x) != np.float else 'Unknown')

In [221]:
# Correct Typos
ndf.loc[ndf['ORF1_sub']=='Pd)', 'ORF1_sub'] = 'Pd'
ndf.loc[ndf['ORF1_sub']=='Pc)', 'ORF1_sub'] = 'Pc'
ndf.loc[ndf['ORF1_sub']=='Pe)', 'ORF1_sub'] = 'Pe'

### Sort out columns depending on their type of data
* with boolean (e.g. yes/no)
* with only strings
* with few integers
* with a range of numbers
* with datetime data

Then we could put columns with fewer unique values in the columns of the pivottable

In [223]:
booleans = ['Outbreak or sporadic case O/S', 'Point source transmission Y/N', 'Mixed infection Y/N', 'Included in II.4 P2 capsid surveillance']
numbers = [ 'Nr of persons affected', 'Nr of persons at risk', 'Nr of cases deceased', 'Nr of cases hospitalized due to infection', 'Nr of cases of ages 0 to 4',
           'Nr of cases of ages 5 to 14', 'Nr of cases of ages 15 to 64', 'Nr of cases of age 65 or older', 'Nr of cases with vomiting', 'Nr of cases with diarrhea',
            'Nr of samples tested', 'Nr of PCR positive samples', 'Nr of PCR negative samples']
geo = [ 'Institute', 'Country', 'Suspected country of infection']
dates = ['Submission Date', 'Last Update', 'Sample Date','Date of first disease','Date of point source transmission',]
ids = ['User', 'Database ID', 'Outbreak Nr']
pathological = ['Source of the sample', 'Specify source', 'Suspected mode of transmission', 'Specify other mode of transmission', 'Food item', 'Specify food item',
 'Setting of the outbreak', 'Specify setting', 'Specify other pathogen(s)']
seq_data =['fasta_id', 'reference_id', 'fragment_begin', 'Genus-Genogroup', 'ORF1', 'ORF1_sub', 'ORF1_variant', 'ORF2', 'ORF2_sub', 'ORF2_variant', 'Reference_sequence_for_positions', 'Cluster']
etc = ['Geo-coded location']

### Saving data

In [224]:
# dataframe
ndf.to_json("noronet_clean.json")
ndf.to_feather("noronet_clean.fr")
# categories
dict_categories = {'geo':geo,
                  'booleans':booleans,
                  'numbers': numbers,
                  'dates': dates,
                   'ids': ids,
                   'pathological': pathological,
                   'seq_data': seq_data,
                   'etc': etc
                  }

with open("categories.json", 'w') as f:
    f.write(json.dumps(dict_categories))


### convert date to datetime

In [211]:
for i in range(len(dates)):
    ndf.loc[:, dates[i]] = pd.to_datetime(ndf[dates[i]])

In [212]:
ndf[dates].head()

,Submission Date,Last Update,Sample Date,Date of first disease,Date of point source transmission
0,2018-04-01,2018-01-04 11:44:22,2016-12-20,2016-12-20,NaT
1,2018-04-01,2018-01-04 11:46:45,2016-12-27,2016-12-23,NaT
2,2018-04-01,2018-01-04 11:48:48,2016-12-21,2016-12-18,NaT
3,2018-04-01,2018-01-04 11:51:12,2016-12-20,2016-12-17,NaT
4,2018-04-01,2018-01-04 11:53:13,2016-12-12,2016-09-12,NaT


### Lets see each categories in detail, and check if we didn't miss any detail

In [213]:
#booleans
for c in booleans:
    print(ndf[c].unique())
ndf[booleans].info()

['O' 'S' nan]
['No' nan 'Yes']
['No' 'Yes' nan]
['No' 'Yes' nan]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3140 entries, 0 to 3139
Data columns (total 4 columns):
 #   Column                                   Non-Null Count  Dtype 
---  ------                                   --------------  ----- 
 0   Outbreak or sporadic case O/S            2166 non-null   object
 1   Point source transmission Y/N            1727 non-null   object
 2   Mixed infection Y/N                      1747 non-null   object
 3   Included in II.4 P2 capsid surveillance  1785 non-null   object
dtypes: object(4)
memory usage: 98.2+ KB


In [214]:
#numbers
dict_numbers_unique_values = { c: len(ndf[c].unique()) for c in numbers}
print("Number of unique values for each column")
for k in dict_numbers_unique_values.keys():
    print(k, "\t",dict_numbers_unique_values[k])

print("\nInformations about the numbers category")
ndf[numbers].info()

Number of unique values for each column
Nr of persons affected 	 76
Nr of persons at risk 	 118
Nr of cases deceased 	 4
Nr of cases hospitalized due to infection 	 24
Nr of cases of ages 0 to 4 	 17
Nr of cases of ages 5 to 14 	 30
Nr of cases of ages 15 to 64 	 39
Nr of cases of age 65 or older 	 52
Nr of cases with vomiting 	 36
Nr of cases with diarrhea 	 38
Nr of samples tested 	 32
Nr of PCR positive samples 	 26
Nr of PCR negative samples 	 21

Informations about the numbers category
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3140 entries, 0 to 3139
Data columns (total 13 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   Nr of persons affected                     1300 non-null   float64
 1   Nr of persons at risk                      412 non-null    float64
 2   Nr of cases deceased                       565 non-null    float64
 3   Nr of cases hospitalized due to 

In [215]:
#geo
dict_unique_values = { c: len(ndf[c].unique()) for c in geo}
print("Number of unique values for each column")
for k in dict_unique_values.keys():
    print(k, "\t",dict_unique_values[k])

print("\nInformations about the geo data")
ndf[geo].info()

Number of unique values for each column
Institute 	 19
Country 	 14
Suspected country of infection 	 6

Informations about the geo data
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3140 entries, 0 to 3139
Data columns (total 3 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   Institute                       3140 non-null   object
 1   Country                         3140 non-null   object
 2   Suspected country of infection  2512 non-null   object
dtypes: object(3)
memory usage: 73.7+ KB


In [216]:
#dates
dict_unique_values = { c: len(ndf[c].unique()) for c in dates}
print("Number of unique values for each column")
for k in dict_unique_values.keys():
    print(k, "\t",dict_unique_values[k])

print("\nInformations about the numbers category")
ndf[dates].info()

Number of unique values for each column
Submission Date 	 150
Last Update 	 2178
Sample Date 	 830
Date of first disease 	 506
Date of point source transmission 	 2

Informations about the numbers category
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3140 entries, 0 to 3139
Data columns (total 5 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   Submission Date                    3140 non-null   datetime64[ns]
 1   Last Update                        3140 non-null   datetime64[ns]
 2   Sample Date                        3140 non-null   datetime64[ns]
 3   Date of first disease              1577 non-null   datetime64[ns]
 4   Date of point source transmission  8 non-null      datetime64[ns]
dtypes: datetime64[ns](5)
memory usage: 122.8 KB


In [217]:
#pathological
dict_unique_values = { c: len(ndf[c].unique()) for c in pathological}
print("Number of unique values for each column")
for k in dict_unique_values.keys():
    print(k, "\t",dict_unique_values[k])

print("\nInformations about the pathological category")
ndf[pathological].info()

Number of unique values for each column
Source of the sample 	 4
Specify source 	 4
Suspected mode of transmission 	 7
Specify other mode of transmission 	 12
Food item 	 10
Specify food item 	 29
Setting of the outbreak 	 12
Specify setting 	 63
Specify other pathogen(s) 	 35

Informations about the pathological category
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3140 entries, 0 to 3139
Data columns (total 9 columns):
 #   Column                              Non-Null Count  Dtype 
---  ------                              --------------  ----- 
 0   Source of the sample                3140 non-null   object
 1   Specify source                      65 non-null     object
 2   Suspected mode of transmission      963 non-null    object
 3   Specify other mode of transmission  31 non-null     object
 4   Food item                           208 non-null    object
 5   Specify food item                   98 non-null     object
 6   Setting of the outbreak             1873 non-null   o

In [218]:
#seq
print("\nInformations about the the sequences")
ndf[seq_data].info()


Informations about the the sequences
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3140 entries, 0 to 3139
Data columns (total 12 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   fasta_id                          3140 non-null   object 
 1   reference_id                      3098 non-null   object 
 2   fragment_begin                    3098 non-null   float64
 3   Genus-Genogroup                   3140 non-null   object 
 4   ORF1                              3140 non-null   object 
 5   ORF1_sub                          3140 non-null   object 
 6   ORF1_variant                      154 non-null    object 
 7   ORF2                              3140 non-null   object 
 8   ORF2_sub                          3140 non-null   object 
 9   ORF2_variant                      940 non-null    object 
 10  Reference_sequence_for_positions  3098 non-null   object 
 11  Cluster                        